In [2]:
import os
import glob
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [ ]:
FOLDER = "Datasets\Dataset2-InstaCartOnlineGroceryBasketAnalysisDataset"   # <-- change this
SAMPLE_SIZE = 1000                    # number of rows sampled per file
METHOD = "random"                     # "random" (reservoir) or "head"
CHUNKSIZE = 20000                     # pandas chunksize for streaming
ENCODING = None                       # e.g. "latin-1" if utf-8 fails
COUNT_ROWS = False                    # whether to count total rows (slow)
OUT_DIR = None                        # e.g. "/tmp/csv_summaries" or None to skip writing
RANDOM_SEED = 42

In [ ]:
# functions: sampling, analysis, and folder processing
if RANDOM_SEED is not None:
    np.random.seed(int(RANDOM_SEED))

def reservoir_sample_csv(path, sample_size=1000, chunksize=20000, encoding=None):
    """
    Stream a CSV in chunks and produce a reservoir sample of rows (as a DataFrame).
    Returns (sample_df, total_rows_seen)
    """
    total = 0
    reservoir = []
    reader = pd.read_csv(path, chunksize=chunksize, iterator=True, dtype=str, encoding=encoding, low_memory=True)
    try:
        for chunk in reader:
            records = chunk.to_dict(orient="records")
            for rec in records:
                total += 1
                if len(reservoir) < sample_size:
                    reservoir.append(rec)
                else:
                    r = np.random.randint(0, total)
                    if r < sample_size:
                        reservoir[r] = rec
    except pd.errors.EmptyDataError:
        return pd.DataFrame(), 0
    except pd.errors.ParserError:
        # fall back to reading header only if parsing fails
        try:
            hdr = pd.read_csv(path, nrows=0, dtype=str, encoding=encoding)
            return pd.DataFrame(columns=hdr.columns), total
        except Exception:
            return pd.DataFrame(), total

    if len(reservoir) == 0:
        try:
            header = pd.read_csv(path, nrows=0, dtype=str, encoding=encoding)
            return pd.DataFrame(columns=header.columns), total
        except Exception:
            return pd.DataFrame(), total

    sample_df = pd.DataFrame(reservoir)
    return sample_df, total

def head_sample_csv(path, sample_size=1000, encoding=None):
    """
    Read the first sample_size rows using pandas.read_csv(nrows=...).
    Returns (sample_df, None)
    """
    try:
        df = pd.read_csv(path, nrows=sample_size, dtype=str, encoding=encoding, low_memory=True)
    except pd.errors.EmptyDataError:
        df = pd.DataFrame()
    except pd.errors.ParserError:
        # try manual engine fallback
        df = pd.read_csv(path, nrows=sample_size, dtype=str, encoding=encoding, engine="python")
    return df, None

def try_cast_numeric(s: pd.Series):
    numeric = pd.to_numeric(s, errors='coerce')
    is_numeric = numeric.notna()
    return numeric, is_numeric

def analyze_sample(df: pd.DataFrame, file_path: str, sample_rows_seen: int=None, count_total_rows=None):
    """
    Return (summary_df, meta_dict) for the sampled DataFrame.
    summary_df: one row per column with key stats.
    meta_dict: file-level metadata.
    """
    if df is None or df.shape[1] == 0:
        return pd.DataFrame(), {"file_path": file_path, "file_size_bytes": os.path.getsize(file_path) if os.path.exists(file_path) else None, "sample_rows_returned": 0, "sample_rows_seen_total": count_total_rows or sample_rows_seen}

    cols = df.columns.tolist()
    summary = []
    n_sample = len(df)

    # normalize NA representation within the sample
    df_clean = df.fillna("").astype(str)

    for col in cols:
        col_series = df_clean[col].replace({"": None})
        missing_mask = col_series.isnull()
        missing = int(missing_mask.sum())
        non_missing = col_series[~missing_mask]
        unique = int(non_missing.nunique(dropna=True))
        top_values = non_missing.value_counts(dropna=True).head(5).to_dict()

        numeric_conv, is_numeric_mask = try_cast_numeric(col_series)
        numeric_count = int(is_numeric_mask.sum())
        numeric_fraction = numeric_count / n_sample if n_sample > 0 else 0.0

        col_summary = {
            "file": os.path.basename(file_path),
            "column": col,
            "sample_size": n_sample,
            "missing": missing,
            "missing_frac": round(missing / n_sample, 6) if n_sample > 0 else None,
            "unique_sampled": unique,
            "top_values_sampled": top_values,
            "numeric_count_in_sample": numeric_count,
            "numeric_fraction_in_sample": round(numeric_fraction, 6),
        }

        if numeric_count >= 1:
            numeric_series = numeric_conv[is_numeric_mask]
            col_summary.update({
                "num_count": int(numeric_series.count()),
                "num_mean": float(numeric_series.mean()) if numeric_series.count() > 0 else None,
                "num_std": float(numeric_series.std(ddof=0)) if numeric_series.count() > 0 else None,
                "num_min": float(numeric_series.min()) if numeric_series.count() > 0 else None,
                "num_25%": float(numeric_series.quantile(0.25)) if numeric_series.count() > 0 else None,
                "num_50%": float(numeric_series.quantile(0.50)) if numeric_series.count() > 0 else None,
                "num_75%": float(numeric_series.quantile(0.75)) if numeric_series.count() > 0 else None,
                "num_max": float(numeric_series.max()) if numeric_series.count() > 0 else None,
            })
        else:
            lengths = non_missing.map(len) if len(non_missing) > 0 else pd.Series(dtype=int)
            col_summary.update({
                "num_count": 0,
                "str_len_min": int(lengths.min()) if len(lengths) > 0 else None,
                "str_len_mean": float(lengths.mean()) if len(lengths) > 0 else None,
                "str_len_max": int(lengths.max()) if len(lengths) > 0 else None,
            })

        summary.append(col_summary)

    summary_df = pd.DataFrame(summary)
    meta = {
        "file_path": file_path,
        "file_size_bytes": os.path.getsize(file_path) if os.path.exists(file_path) else None,
        "sample_rows_returned": n_sample,
        "sample_rows_seen_total": count_total_rows if count_total_rows is not None else sample_rows_seen
    }
    return summary_df, meta

def process_folder(folder, sample_size=1000, method='random', chunksize=20000, encoding=None, count_rows=False, out_dir=None, show_table=True):
    """
    Process CSV files in folder; return (combined_summary_df, metadata_list).
    Prints a compact table per file (unless show_table=False).
    """
    pattern = os.path.join(folder, "*.csv")
    files = sorted(glob.glob(pattern))
    if not files:
        raise FileNotFoundError(f"No CSV files found in folder: {folder}")

    all_summaries = []
    all_meta = []

    for f in files:
        print(f"\n--- Processing: {os.path.basename(f)} ---")
        try:
            if method == 'head':
                sample_df, total_seen = head_sample_csv(f, sample_size=sample_size, encoding=encoding)
                total_rows = None
                if count_rows:
                    with open(f, "rb") as fh:
                        total_rows = sum(1 for _ in fh) - 1
            else:
                sample_df, total_seen = reservoir_sample_csv(f, sample_size=sample_size, chunksize=chunksize, encoding=encoding)
                total_rows = total_seen

            if sample_df is None or sample_df.shape[1] == 0:
                print("  (no columns or empty file)")
                continue

            summary_df, meta = analyze_sample(sample_df, f, sample_rows_seen=len(sample_df), count_total_rows=total_rows)
            all_summaries.append(summary_df)
            all_meta.append(meta)

            display_cols = ["column", "sample_size", "missing", "missing_frac", "unique_sampled", "numeric_fraction_in_sample"]
            if show_table:
                display(HTML(summary_df[display_cols].to_html(index=False)))

            if out_dir:
                os.makedirs(out_dir, exist_ok=True)
                out_path = os.path.join(out_dir, os.path.basename(f) + ".summary.csv")
                summary_df.to_csv(out_path, index=False)
                print(f"  Summary written to: {out_path}")

        except Exception as e:
            print(f"  ERROR processing {f}: {repr(e)}")

    if all_summaries:
        combined = pd.concat(all_summaries, ignore_index=True)
        if out_dir:
            combined_out = os.path.join(out_dir, "combined_summary.csv")
            combined.to_csv(combined_out, index=False)
            print(f"\nCombined summary written to: {combined_out}")
        return combined, all_meta
    else:
        return pd.DataFrame(), all_meta


In [ ]:
combined_summary, metadata = process_folder(
    folder=FOLDER,
    sample_size=SAMPLE_SIZE,
    method=METHOD,
    chunksize=CHUNKSIZE,
    encoding=ENCODING,
    count_rows=COUNT_ROWS,
    out_dir=OUT_DIR,
    show_table=True
)

# display combined summary (compact)
if not combined_summary.empty:
    display(HTML(combined_summary[["file","column","sample_size","missing","missing_frac","unique_sampled","numeric_fraction_in_sample"]].to_html(index=False)))
else:
    print("No summary generated.")


--- Processing: aisles.csv ---


column,sample_size,missing,missing_frac,unique_sampled,numeric_fraction_in_sample
aisle_id,134,0,0.0,134,1.0
aisle,134,0,0.0,134,0.0



--- Processing: departments.csv ---


column,sample_size,missing,missing_frac,unique_sampled,numeric_fraction_in_sample
department_id,21,0,0.0,21,1.0
department,21,0,0.0,21,0.0



--- Processing: order_products__prior.csv ---


...


--- Processing: orders.csv ---


column,sample_size,missing,missing_frac,unique_sampled,numeric_fraction_in_sample
order_id,1000,0,0.000,1000,1.000
user_id,1000,0,0.000,991,1.000
eval_set,1000,0,0.000,3,0.000
order_number,1000,0,0.000,84,1.000
order_dow,1000,0,0.000,7,1.000
order_hour_of_day,1000,0,0.000,23,1.000
days_since_prior_order,1000,65,0.065,31,0.935



--- Processing: products.csv ---


column,sample_size,missing,missing_frac,unique_sampled,numeric_fraction_in_sample
product_id,1000,0,0.0,1000,1.0
product_name,1000,0,0.0,1000,0.0
aisle_id,1000,0,0.0,131,1.0
department_id,1000,0,0.0,21,1.0


file,column,sample_size,missing,missing_frac,unique_sampled,numeric_fraction_in_sample
aisles.csv,aisle_id,134,0,0.000,134,1.000
aisles.csv,aisle,134,0,0.000,134,0.000
departments.csv,department_id,21,0,0.000,21,1.000
departments.csv,department,21,0,0.000,21,0.000
order_products__prior.csv,order_id,1000,0,0.000,998,1.000
order_products__prior.csv,product_id,1000,0,0.000,758,1.000
order_products__prior.csv,add_to_cart_order,1000,0,0.000,40,1.000
order_products__prior.csv,reordered,1000,0,0.000,2,1.000
order_products__train.csv,order_id,1000,0,0.000,994,1.000
order_products__train.csv,product_id,1000,0,0.000,802,1.000
